In [1]:
#r "nuget:Microsoft.ML, 1.5.2"
#r "nuget:Microsoft.AspNetCore.Html.Abstractions, 2.2.0"
#r "System.Collections.Immutable.dll"

using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using System.Collections.Immutable;
using XPlot.Plotly;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.2

Installed package Microsoft.AspNetCore.Html.Abstractions version 2.2.0

## Class IrisData:

In [2]:
public class IrisData
{
    [LoadColumn(0)]
    public float SepalLength;

    [LoadColumn(1)]
    public float SepalWidth;

    [LoadColumn(2)]
    public float PetalLength;

    [LoadColumn(3)]
    public float PetalWidth;

    [LoadColumn(4)]
    public string Label = "";

    public void SetProperty(string property, float value)
    {
        switch (property)
        {
            case "SepalLength":
                SepalLength = value;
                break;
            case "SepalWidth":
                SepalWidth = value;
                break;
            case "PetalLength":
                PetalLength = value;
                break;
            case "PetalWidth":
                PetalWidth = value;
                break;
        }
    }
}

## Class LabelPrediction:

In [3]:
public class LabelPrediction
{
    [ColumnName("PredictedLabel")]
    public uint PredictedLabelId = 0;
}

## Form train and test data:

In [4]:
static readonly string _dataPath = "iris.csv";
var mlContext = new MLContext(seed: 0);
IDataView dataView = mlContext.Data.LoadFromTextFile<IrisData>(
    _dataPath, hasHeader: false, separatorChar: ','
    );
dataView = mlContext.Data.ShuffleRows(dataView, seed: 42);
var ttSet = mlContext.Data.TrainTestSplit(dataView, testFraction: 0.2);

## Graphics:

In [5]:
public void Scatter(bool SorP) 
{
    string name = SorP ? "Sepal" : "Petal";
    List<double>[] SP = new List<double>[]
    {
        dataView.Preview(150).ColumnView[SorP ? 0 : 2].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Setosa").ToList(),
        dataView.Preview(150).ColumnView[SorP ? 1 : 3].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Setosa").ToList(),
        dataView.Preview(150).ColumnView[SorP ? 0 : 2].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Versicolor").ToList(),
        dataView.Preview(150).ColumnView[SorP ? 1 : 3].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Versicolor").ToList(),
        dataView.Preview(150).ColumnView[SorP ? 0 : 2].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Virginica").ToList(),
        dataView.Preview(150).ColumnView[SorP ? 1 : 3].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Virginica").ToList(),
    };
    var SetosaScatter = new Graph.Scatter()
    {
        x = SP[0],
        y = SP[1],
        mode = "markers",
        name = "Setosa",
        marker =
            new Graph.Marker
            {
                color = "rgb(164, 194, 244)",
                size = 12,
            }
    };
    var VersicolorScatter = new Graph.Scatter()
    {
        x = SP[2],
        y = SP[3],
        mode = "markers",
        name = "Versicolor",
        marker =
            new Graph.Marker
            {
                color = "rgb(142, 124, 195)",
                size = 12,
            }
    };
    var VirginicaScatter = new Graph.Scatter()
    {
        x = SP[4],
        y = SP[5],
        mode = "markers",
        name = "Virginica",
        marker =
            new Graph.Marker
            {
                color = "rgb(255, 217, 102)",
                size = 12,
            }
    };
    var irisChart = Chart.Plot(new[]
        {
            SetosaScatter,
            VersicolorScatter,
            VirginicaScatter
        }
    );
    irisChart.WithTitle(name);
    irisChart.WithXTitle($"{name} Length");
    irisChart.WithYTitle($"{name} Width");
    display(irisChart);
}

var AreaPlotSepal = Chart.Plot(new Graph.Contour()
{
    x = dataView.Preview(150).ColumnView[0].Values,
    y = dataView.Preview(150).ColumnView[1].Values,
    z = dataView.Preview(150).ColumnView[4].Values.Select(item=>item.ToString() == "Setosa" ? 1 : item.ToString() == "Versicolor" ? 2 : 3)
});
var AreaPlotPetal = Chart.Plot(new Graph.Contour()
{
    x = dataView.Preview(150).ColumnView[2].Values,
    y = dataView.Preview(150).ColumnView[3].Values,
    z = dataView.Preview(150).ColumnView[4].Values.Select(item=>item.ToString() == "Setosa" ? 1 : item.ToString() == "Versicolor" ? 2 : 3)
});
AreaPlotSepal.WithTitle("Contour Plot Sepal");
AreaPlotPetal.WithTitle("Contour Plot Petal");
Scatter(true);
display(AreaPlotSepal);
Scatter(false);
display(AreaPlotPetal);

In [6]:
public void Scatter3d(int a, int b, int c)
{
    var SetosaScatter = new Graph.Scatter3d()
    {
        x = dataView.Preview(150).ColumnView[a].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Setosa").ToList(),
        y = dataView.Preview(150).ColumnView[b].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Setosa").ToList(),
        z = dataView.Preview(150).ColumnView[c].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Setosa").ToList(),
        mode = "markers",
        name = "Setosa",
        marker =
            new Graph.Marker
            {
                color = "rgb(164, 194, 244)",
                size = 7,
            }
    };
    var VersicolorScatter = new Graph.Scatter3d()
    {
        x = dataView.Preview(150).ColumnView[a].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Versicolor").ToList(),
        y = dataView.Preview(150).ColumnView[b].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Versicolor").ToList(),
        z = dataView.Preview(150).ColumnView[c].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Versicolor").ToList(),
        mode = "markers",
        name = "Versicolor",
        marker =
            new Graph.Marker
            {
                color = "rgb(142, 124, 195)",
                size = 7,
            }
    };
    var VirginicaScatter = new Graph.Scatter3d()
    {
        x = dataView.Preview(150).ColumnView[a].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Virginica").ToList(),
        y = dataView.Preview(150).ColumnView[b].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Virginica").ToList(),
        z = dataView.Preview(150).ColumnView[c].Values.Select(item => Convert.ToDouble(item))
        .Where((item, i)=>dataView.Preview(150).ColumnView[4].Values[i].ToString()=="Virginica").ToList(),
        mode = "markers",
        name = "Virginica",
        marker =
            new Graph.Marker
            {
                color = "rgb(255, 217, 102)",
                size = 7,
            }
    };
    var irisChart = Chart.Plot(new[]
        {
            SetosaScatter,
            VersicolorScatter,
            VirginicaScatter
        }
    );

    display(irisChart);
}
Scatter3d(0,1,2);
Scatter3d(0,1,3);
Scatter3d(1,2,3);
Scatter3d(0,2,3);

## Pipeline:

In [7]:
string featuresColumnName = "Features";
var pipeline = mlContext.Transforms
    .Concatenate(featuresColumnName, "SepalLength", "SepalWidth", "PetalLength", "PetalWidth")
    .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "Label",
    inputColumnName: "Label"))
    .Append(mlContext.MulticlassClassification.Trainers.LbfgsMaximumEntropy(labelColumnName:"Label",
    featureColumnName:featuresColumnName));

## Model:

In [8]:
var model = pipeline.Fit(ttSet.TrainSet);
var predictions = model.Transform(ttSet.TrainSet);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions);
var predictor = mlContext.Model.CreatePredictionEngine<IrisData, LabelPrediction>(model);
Console.WriteLine($"Accuracy: {metrics.MacroAccuracy}\n\n");

Accuracy: 0,9655870445344129




## Test:

In [9]:
int mistakesCount = 0;
foreach (var item in ttSet.TestSet.Preview(150).RowView)
{
    var sample = new IrisData();
    foreach (var property in item.Values)
    {
        if(property.Key != "Label")
        {
            Console.Write(property + " ");
            sample.SetProperty(property.Key, (float)property.Value);
        }
    }
    var prediction = predictor.Predict(sample);
    Console.WriteLine("\nReal label:" + item.Values[^1].Value);
    string predictedLabel = prediction.PredictedLabelId == 1 ? "Versicolor" :
        prediction.PredictedLabelId == 2 ? "Setosa":"Virginica";
    Console.WriteLine($"Predicted label: {predictedLabel}\n");
    mistakesCount += predictedLabel == item.Values[^1].Value.ToString() ? 0 : 1;
}
Console.WriteLine($"Mistakes: {mistakesCount}");
Console.WriteLine($"Accuracy: {(ttSet.TestSet.Preview(150).RowView.Count() - mistakesCount) / ttSet.TestSet.Preview(150).RowView.Count()}");

[SepalLength, 6] [SepalWidth, 3,4] [PetalLength, 4,5] [PetalWidth, 1,6] 
Real label:Versicolor
Predicted label: Versicolor

[SepalLength, 5,5] [SepalWidth, 2,5] [PetalLength, 4] [PetalWidth, 1,3] 
Real label:Versicolor
Predicted label: Versicolor

[SepalLength, 5,7] [SepalWidth, 2,6] [PetalLength, 3,5] [PetalWidth, 1] 
Real label:Versicolor
Predicted label: Versicolor

[SepalLength, 6,5] [SepalWidth, 2,8] [PetalLength, 4,6] [PetalWidth, 1,5] 
Real label:Versicolor
Predicted label: Versicolor

[SepalLength, 5,6] [SepalWidth, 3] [PetalLength, 4,1] [PetalWidth, 1,3] 
Real label:Versicolor
Predicted label: Versicolor

[SepalLength, 4,6] [SepalWidth, 3,2] [PetalLength, 1,4] [PetalWidth, 0,2] 
Real label:Setosa
Predicted label: Setosa

[SepalLength, 7,1] [SepalWidth, 3] [PetalLength, 5,9] [PetalWidth, 2,1] 
Real label:Virginica
Predicted label: Virginica

[SepalLength, 7,7] [SepalWidth, 3] [PetalLength, 6,1] [PetalWidth, 2,3] 
Real label:Virginica
Predicted label: Virginica

[SepalLength, 4,